In [1]:
import wave
import random
import librosa
import os
import torch
from tqdm import tqdm
import numpy as np
from torch import nn
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def extract_feature(file_name, mfcc_count=40):

    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=mfcc_count).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    chroma_cens = librosa.feature.chroma_cens(y=X, sr=sample_rate, n_chroma=20).mean(axis=1)
    tempo = librosa.feature.rhythm.tempogram(y=X, sr=sample_rate, win_length=50).mean(axis=1)
    
    return np.concatenate((mfccs, chroma, mel, contrast, chroma_cens, tempo))

In [3]:
def write_wav(frames, name, sample_rate=16000):
    
    f = wave.open(name, 'wb')
    f.setnchannels(1)
    f.setsampwidth(2)
    f.setframerate(sample_rate)
    f.setnframes(int(len(frames)))
    f.writeframes(frames)
    f.close()

In [4]:
def cut_wav(frames):
    
    maximum = max(frames)
    
    for i in range(len(frames)):
        if frames[i] > maximum/13:
            start = i
            break
    
    for i in range(len(frames)):
        if frames[::-1][i] > maximum/13:
            stop = i
            break
            
    return start, len(frames) - stop

In [5]:
files_classes = {}
X = []
y = []

with open("data_v_7_stc/meta/meta.txt") as f:
    for line in f:
        file_name = line.split()[0]
        file_class = line.split()[-1]
        files_classes[file_name] = file_class
        
unique_classes = ['tool', 'keyboard', 'door', 'knocking_door', 'bags', 'speech', 'background', 'ring']
files_class_numbers = {i: unique_classes.index(files_classes[i]) for i in files_classes.keys()}

In [6]:
def cut_files(input_path, output_path, check_speech=False):

    for i in tqdm(os.listdir(path=input_path)):
        
        if check_speech and files_classes[i] == "speech":
            continue

        file = wave.open(input_path + i)
        sample_rate = file.getframerate()
        frames = file.readframes(file.getnframes())
        frames_int16 = (np.frombuffer(frames, dtype=np.int16))
        start, stop = cut_wav(frames_int16)
        write_wav(frames[start * 2 : stop * 2], output_path + i, sample_rate)

In [7]:
os.makedirs("new_data")
os.makedirs("new_test_data")
cut_files("data_v_7_stc/audio/", "new_data/", check_speech=True)
cut_files("data_v_7_stc/test/", "new_test_data/")

100%|██████████| 610/610 [00:23<00:00, 25.94it/s]


In [8]:
speech_files = [i for i in files_classes.keys() if files_classes[i] == "speech"]
speech_data = b""

for i in tqdm(speech_files):
    if i in os.listdir("data_v_7_stc/audio/"):
        f = wave.open("data_v_7_stc/audio/"+i)
        speech_data += f.readframes(f.getnframes())
        
    
    
fps = 16000*2
speech_files_count = len(speech_data)/fps//6
speech_files_count = int(speech_files_count)
    
os.makedirs("new_speech")

for i in tqdm(range(speech_files_count)):
    write_wav(speech_data[i*fps*6:(i+1)*fps*6], "new_speech/test_speech{}.wav".format(i))

100%|██████████| 1653/1653 [00:00<00:00, 8533.95it/s]


In [ ]:
X = []
y = []
X_file_names = []
for i in tqdm(os.listdir(path="new_data/")):
    try:
        X_file_names.append(i)
        X.append(extract_feature("new_data/"+i, mfcc_count=60))
        y.append(files_class_numbers[i])
    except:
        pass


X = [X[i] for i in range(len(X)) if y[i] is not 5]
X_file_names = [X_file_names[i] for i in range(len(X)) if y[i] is not 5]
y = [i for i in y if i is not 5]

for i in tqdm(os.listdir(path="new_speech/")):
    print(i)
    try:    
        X_file_names.append(i)
        X.append(extract_feature("new_speech/"+i, mfcc_count=60))
        y.append(unique_classes.index("speech"))
    except:
        pass
    
    
X_final = []
X_final_names = []

for i in tqdm(os.listdir(path="new_test_data/")):
    try:    
        X_final.append(extract_feature("new_test_data/"+i, mfcc_count=60))
        X_final_names.append(i)
    except:
        pass

  0%|          | 0/1653 [00:00<?, ?it/s]

test_speech899.wav


  0%|          | 1/1653 [00:00<06:42,  4.10it/s]

test_speech181.wav


  0%|          | 2/1653 [00:00<06:51,  4.02it/s]

...

 93%|█████████▎| 1530/1653 [07:15<00:34,  3.51it/s]

test_speech1538.wav


 93%|█████████▎| 1531/1653 [07:15<00:34,  3.51it/s]

test_speech114.wav


 93%|█████████▎| 1532/1653 [07:15<00:34,  3.52it/s]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.05, shuffle = True)

In [ ]:
class MyDataset(Dataset): 

    def __init__(self, X, y):
        
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [ ]:
train_dataset = MyDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=len(X_train), shuffle=True)

test_dataset = MyDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=len(X_test), shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear = nn.Sequential(
                nn.Linear(len(X_scaled[0]), 500),
                nn.BatchNorm1d(500),
                nn.ReLU(),
                nn.Dropout(0.45),
                nn.Linear(500, 1500),
                nn.BatchNorm1d(1500),
                nn.ReLU(),
                nn.Dropout(0.45),
                nn.Linear(1500, 200),
                nn.BatchNorm1d(200),
                nn.ReLU(),
                nn.Dropout(0.45),
                nn.Linear(200, 8),
                nn.Softmax())
        
        self.loss_test = []
        self.loss_train = []

        self.acc_test = []
        self.acc_train = []

    
    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
net = Net().to(device)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:


def train_model(net, epoch_count, train_loader, test_loader, learning_rate=0.00001):
    opt = torch.optim.Adam(net.parameters(), lr=learning_rate)
    for i in range(epoch_count):


        #test
        running_loss = []
        running_acc = []
        
        for sample, label in test_loader:
            data = sample.to(device)
            label = label.to(device)
            net.eval()
            out = net(data)
            _, predicted = torch.max(out, 1)
            loss_t = loss_fn(out, label)
            running_loss.append(loss_t.item())
            running_acc.append(sum((predicted == label).tolist())/len(predicted))

        print("---------- \n test")
        acc = np.mean(running_acc)
        loss_ = np.mean(running_loss)
        net.acc_test.append(acc)
        print(acc)
        net.loss_test.append(loss_)
        
        
        #train
        running_loss = []
        running_acc = []
        
        
        for sample, label in train_loader:
            #augmentation_mask = torch.normal(torch.ones(sample.shape), torch.ones(sample.shape)*0.2)
            #sample = sample*augmentation_mask
            data = (sample).to(device)
            label = label.to(device)
            net.train()
            net.zero_grad()
            out = net(data)
            loss = loss_fn(out, label)
            _, predicted = torch.max(out, 1)
            loss.backward()
            opt.step()  
            running_loss.append(loss.item())
            running_acc.append(sum((predicted == label).tolist())/len(predicted))
        print("---------- \n train")
        acc = np.mean(running_acc)
        loss_ = np.mean(running_loss)
        net.acc_train.append(acc)
        print(acc)
        net.loss_train.append(loss_)

        if i%10 == 0:
            plt.figure(figsize=(8,7))
            plt.subplot(2,1,1)
            plt.title("Loss")
            plt.plot(net.loss_test, color="orange", label="test")
            plt.plot(net.loss_train, color="blue", label="train")
            plt.legend()
            plt.grid()
            plt.subplot(2,1,2)
            plt.title("Accuracy")
            plt.plot(net.acc_test, color="orange", label="test")
            plt.plot(net.acc_train, color="blue", label="train")
            plt.legend()
            plt.grid()
            plt.show()

In [ ]:
train_model(net, 2500, train_loader, test_loader, learning_rate=0.00003)

In [ ]:
tmp_net = net
tmp_net.eval()
X_final_scale = scaler.transform(X_final)
out = tmp_net((torch.FloatTensor(X_final_scale)).to(device))
_, predicted = torch.max(out, 1)
class_prob = list(map(max, out.tolist()))
ans = []
for i in range(len(predicted)):
    ans.append((X_final_names[i], str(class_prob[i])[:6], unique_classes[predicted[i].item()]))
    
for i in sorted(ans, key=lambda x: x[0]):
    print (*i, sep="\t")
    

In [ ]:
ans = ["\t".join(i)+"\n" for i in sorted(ans, key=lambda x: x[0])]

with open("result.txt", "w") as f:
    f.writelines(ans)